# SHAP feature selection #
## Code to select feature with combination of LightGBM and SHAP ##

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_folds = 5 # this sets global setting of which how many bootstraps to use

#time_budget_seconds = 200

n_trials = 5

threads = 1

lgb_params = {
    'boosting_type': "dart",#'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'max_leaves': 20,
    'max_depth': 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'min_data_in_leaf': 6,
    'lambda_l1': 0.9,
    'lambda_l2': 0.9,
    "verbose": -1
}

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import *
from yspecies.partition import DataPartitioner, FeatureSelection, DataExtractor
from yspecies.selection import ShapSelector, ModelFactory

In [4]:
from yspecies.tuning import * #for hyperparameter tuning

In [5]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [8]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12340)",12340,39,445,"(12340, 2)","(40, 19)"


## Setting up SHAP selection pipeline ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [9]:
from sklearn.pipeline import Pipeline

selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])


# Tuning started #

In [10]:
ext = Pipeline([
    ('extractor', DataExtractor(selection)), # to extract the data required for ML from the dataset
    ("partitioner", DataPartitioner(nfolds = number_of_folds, nhold_out = 1, species_in_validation=2, not_validated_species = ["Homo_sapiens"]))
])

In [11]:
parts = ext.fit_transform(data)
parts.indexes

[[0,
  260,
  5,
  265,
  10,
  270,
  15,
  275,
  20,
  25,
  30,
  35,
  45,
  50,
  55,
  60,
  65,
  325,
  70,
  330,
  335,
  340,
  85,
  345,
  90,
  350,
  95,
  355,
  100,
  360,
  105,
  108,
  109,
  110,
  111,
  112,
  113,
  365,
  370,
  375,
  380,
  385,
  390,
  395,
  140,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  400,
  405,
  410,
  155,
  415,
  160,
  420,
  165,
  425,
  170,
  430,
  175,
  435,
  180,
  440,
  185,
  190,
  200,
  205,
  210,
  215,
  220,
  225,
  230,
  235,
  240,
  250,
  255],
 [256,
  1,
  261,
  6,
  266,
  11,
  271,
  16,
  276,
  21,
  277,
  278,
  279,
  280,
  26,
  281,
  282,
  283,
  284,
  31,
  285,
  36,
  46,
  51,
  56,
  61,
  321,
  66,
  326,
  71,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  331,
  336,
  341,
  86,
  346,
  91,
  351,
  96,
  356,
  101,
  361,
  106,
  366,
  371,
  376,
  381,
  386,
  391,
  396,
  141,
  401,
  406,
  151,
  411,
  156,
  416,
  161,
  421,
  166,
  426,
  171,
  431,
  176,
  436,
  181,
  441,
  186,
  191,
  196,
  201,
  206,
  211,
  216,
  221,
  226,
  231,
  236,
  241,
  251],
 [257,
  2,
  262,
  7,
  267,
  12,
  272,
  17,
  22,
  27,
  286,
  287,
  32,
  288,
  289,
  290,
  291,
  37,
  292,
  293,
  294,
  295,
  296,
  297,
  298,
  299,
  47,
  300,
  301,
  302,
  303,
  52,
  304,
  305,
  306,
  307,
  57,
  308,
  309,
  310,
  311,
  62,
  312,
  313,
  314,
  315,
  67,
  316,
  317,
  318,
  319,
  72,
  320,
  322,
  327,
  332,
  372,
  337,
  82,
  342,
  87,
  347,
  92,
  352,
  97,
  357,
  102,
  362,
  107,
  367,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  377,
  382,
  387,
  392,
  397,
  142,
  402,
  407,
  152,
  412,
  157,
  417,
  162,
  422,
  167,
  427,
  172,
  432,
  177,
  437,
  182,
  442,
  187,
  192,
  197,
  207,
  212,
  217,
  222,
  227,
  232,
  237,
  252],
 [258,
  3,
  263,
  8,
  268,
  13,
  273,
  18,
  23,
  28,
  33,
  38,
  43,
  48,
  53,
  58,
  63,
  323,
  68,
  328,
  333,
  338,
  83,
  343,
  88,
  348,
  93,
  353,
  98,
  358,
  103,
  363,
  368,
  373,
  378,
  383,
  388,
  393,
  138,
  398,
  143,
  403,
  408,
  153,
  413,
  158,
  418,
  163,
  423,
  168,
  428,
  173,
  433,
  178,
  438,
  183,
  443,
  188,
  193,
  194,
  195,
  198,
  202,
  203,
  208,
  213,
  218,
  223,
  228,
  233,
  238,
  242,
  243,
  244,
  245,
  246,
  247,
  248,
  253],
 [259,
  4,
  264,
  9,
  269,
  14,
  274,
  19,
  24,
  29,
  34,
  39,
  40,
  41,
  42,
  44,
  49,
  54,
  59,
  64,
  324,
  69,
  329,
  334,
  339,
  84,
  344,
  89,
  349,
  94,
  354,
  99,
  359,
  104,
  364,
  369,
  374,
  379,
  126,
  127,
  384,
  129,
  128,
  130,
  131,
  389,
  134,
  132,
  133,
  135,
  394,
  139,
  136,
  137,
  399,
  404,
  409,
  154,
  414,
  159,
  419,
  164,
  424,
  169,
  429,
  174,
  434,
  179,
  439,
  184,
  444,
  189,
  199,
  204,
  209,
  214,
  219,
  224,
  229,
  234,
  239,
  249,
  254]]

In [12]:
assert (len(parts.cv_merged_index) + len(parts.hold_out_merged_index)) == data.samples.shape[0], "cv and hold out should be same as samples number"
assert parts.nhold_out ==1 and parts.hold_out_partition_indexes == [parts.indexes[4]], "checking that hold_out is computed in a right way"

### Crossvalidator ###

In [13]:
cv = CrossValidator()

In [14]:
eval = cv.fit(parts)
eval

[500]	cv_agg's l1: 0.554414 + 0.0125082	cv_agg's l2: 1.28925 + 0.221644	cv_agg's huber: 0.285808 + 0.0149197


{'l1-mean': [17.31925780117958,
  16.303308214205046,
  15.338217645838915,
  14.45122600594176,
  13.639103011515155,
  12.84083047636519,
  12.102137914208727,
  12.435484637446367,
  11.702066350233594,
  11.01828486277449,
  10.376041701585018,
  10.624532245109767,
  9.972547851339272,
  9.39007077843564,
  8.876915466028786,
  8.375130576063915,
  7.927164696159404,
  7.467477709052838,
  7.0532255383121925,
  6.672888188134924,
  6.829904843075442,
  6.46338973368781,
  6.118645922682257,
  5.80439746000099,
  5.500450873124498,
  5.208520108165615,
  4.916891682521895,
  5.068817384633363,
  4.795359757743001,
  4.549304627384016,
  4.657355731473999,
  4.427884552953107,
  4.206253049754837,
  3.9826639719349926,
  4.16132559245803,
  4.477075892063019,
  4.255625745392465,
  4.042996850058896,
  3.8375420885340956,
  3.851071687520922,
  4.058849051785816,
  3.8474999373851566,
  4.013079667950053,
  3.80776305720201,
  3.61963121484417,
  3.7879579572316024,
  3.587267255464

## General Tuner ##

In [14]:
tuner = GeneralTuner(n_trials = n_trials,n_jobs = threads)

In [15]:
'''previos best
{'boosting_type': 'gbdt', 
'lambda_l1': 0.3145968178821861, 
'lambda_l2': 1.7762332630741793, 
'max_leaves': 28, 'max_depth': 3, 
'feature_fraction': 0.6200033629372566, 
'bagging_fraction': 0.7755450899453493, 
'learning_rate': 0.16696041908799225, 
'min_data_in_leaf': 7}
'''

"previos best\n{'boosting_type': 'gbdt', \n'lambda_l1': 0.3145968178821861, \n'lambda_l2': 1.7762332630741793, \n'max_leaves': 28, 'max_depth': 3, \n'feature_fraction': 0.6200033629372566, \n'bagging_fraction': 0.7755450899453493, \n'learning_rate': 0.16696041908799225, \n'min_data_in_leaf': 7}\n"

In [ ]:
best_params = tuner.fit(parts)
best_params

[500]	cv_agg's l1: 0.0852309 + 0.0105691	cv_agg's l2: 0.0249362 + 0.00264861	cv_agg's huber: 0.0124681 + 0.0013243


[I 2020-08-09 07:01:28,290] Trial 0 finished with value: 0.01246809401462032 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 2.218167362588476, 'lambda_l2': 1.7793098908403413, 'max_leaves': 37, 'max_depth': 3, 'feature_fraction': 0.9777925386557023, 'bagging_fraction': 0.642329732644177, 'learning_rate': 0.0732416013616119, 'min_data_in_leaf': 8}. Best is trial 0 with value: 0.01246809401462032.


[500]	cv_agg's l1: 0.026211 + 0.00471658	cv_agg's l2: 0.00860235 + 0.00398866	cv_agg's huber: 0.00429798 + 0.00199129


[I 2020-08-09 07:02:33,875] Trial 1 finished with value: 0.0042979796872140135 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 0.9314657319119465, 'lambda_l2': 0.028317772146111912, 'max_leaves': 20, 'max_depth': 5, 'feature_fraction': 0.9325132736881769, 'bagging_fraction': 0.7089381913711618, 'learning_rate': 0.11224244027538424, 'min_data_in_leaf': 5}. Best is trial 1 with value: 0.0042979796872140135.


In [ ]:
best_params

# Regular code #

In [10]:
#mass = selection.replace(to_predict = "mass_g")
#gestation = selection.replace(to_predict = "mass_g")

In [10]:
def prepare_pipeline(selection: FeatureSelection, folds: int = number_of_folds, model_parameters = lgb_params.copy(), seed: int = None, select_by_gain: bool = True) -> Pipeline: 
    params = model_parameters.copy()
    if seed is not None:
        params["seed"] = seed
        params["drop_seed"] = seed        
    return Pipeline([
    ('extractor', DataExtractor(selection)), # to extract the data required for ML from the dataset
    ("partitioner", DataPartitioner(folds = folds, species_in_validation=2, not_validated_species = ["Homo_sapiens"])), # to partition it according to sorted stratification
    ("shap_computation", ShapSelector(ModelFactory(parameters = params), select_by_gain = select_by_gain))] # to train lightGBM and do feature selection
    )

# First stage selection #

## Maximum lifespan ##

In [11]:
lifespan_pipeline = prepare_pipeline(selection)

In [ ]:
lifespan_gestation =  prepare_pipeline(selection)

In [12]:
lifespan_pipeline_shap = prepare_pipeline(selection, select_by_gain = False)

## Instability problem #

In [13]:
#I commented the seed to show how unstable things can be with multiple runs
#SEED = 42
#random.seed(SEED)
#np.random.seed(seed=SEED)

In [14]:
pipe = lifespan_pipeline

In [ ]:
results1 = pipe.fit_transform(data)
results1

Early stopping is not available in dart mode


In [ ]:
results2 = pipe.fit_transform(data)
results2

In [ ]:
results3 = pipe.fit_transform(data)
results3

## Ploting SHAP values ##

In [ ]:
results1.plot(max_display=100)

In [ ]:
results2.plot(max_display=100)

In [ ]:
results3.plot(max_display=100)

## Intersections ##

In [ ]:
print(results1.selected.shape[0])
print(results2.selected.shape[0])
print(results3.selected.shape[0])

In [ ]:
print("genes same in all 3 runs")
results1.selected.index.intersection(results2.selected.index).intersection(results3.selected.index).shape[0]

## SHAP details display ##

In [18]:
results1.selected_extended

,symbol,gain_score_to_lifespan,kendall_tau_to_lifespan,SRR5961830,SRR5885325,SRR306843,SRR306844,SRR306845,SRR306841,SRR306840,SRR306839,SRR787277,SRR1521445,SRR3715877,SRR5008362,SRR5120939,SRR5120940,SRR8702484,SRR5961972,SRR5961875,SRR306842,SRR306809,SRR306808,SRR306801,SRR649365,SRR306807,SRR306806,SRR306805,SRR306804,SRR306803,SRR306802,SRR306800,SRR1510173,SRR2040586,SRR1758922,SRR1758916,SRR1758921,DRR031591,SRR306793,SRR306791,SRR636900,SRR636850,SRR636945,SRR306834,SRR306826,SRR306827,SRR306829,SRR306832,SRR306835,SRR306828,SRR306831,SRR306833,SRR306836,SRR223519,SRR223518,SRR223520,SRR223521,SRR299126,SRR1047652,SRR1758954,SRR223512,SRR223514,SRR223515,SRR1758941,SRR1758942,SRR1758943,SRR1758957,SRR1759005,SRR1759006,SRR1759007,SRR1759011,SRR1759012,SRR2308103,SRR636978,SRR636932,SRR636888,SRR636977,SRR636933,SRR636887,ERR1331676,SRR306395,SRR306402,SRR306396,SRR306398,SRR306399,SRR306401,SRR2124226,SRR2754983,SRR1048140,SRR2757329,SRR1048142,ERR2716205,ERR2716206,ERR2716207,ERR2716208,SRR1334837,SRR636856,SRR636950,ERR1331678,ERR1331679,SRR636854,SRR636855,SRR636904,SRR636905,SRR636906,SRR636948,SRR636949,SRR3160061,SRR653846,SRR3160008,SRR3160018,SRR3160041,SRR3160052,SRR636937,SRR636938,SRR636842,SRR636843,SRR636892,SRR636893,SRR1300768,SRR1300767,SRR1300766,SRR1300765,SRR1300763,SRR1758979,SRR1758978,SRR1758981,SRR1758985,SRR1758984,SRR1758983,SRR1758982,SRR1758977,SRR5190444,SRR5190446,SRR3109718,SRR3109717,ERR489283,ERR489282,ERR2075986,ERR2075792,ERR2075082,ERR2075072,ERR2074874,SRR6206918,SRR6206908,SRR1205998,SRR8708135,SRR1205223,SRR1205218,SRR1205138,SRR1205222,SRR5190453,SRR489494,SRR1822406,SRR5190441,SRR9024755,SRR9024741,SRR9024747,SRR9024746,SRR9024753,SRR4444968,SRR4444969,SRR3109709,SRR3109705,SRR636935,SRR636934,SRR636840,SRR636839,ERR1331716,SRR1981987,SRR1981981,SRR4444970,SRR4444971,SRR3109710,SRR1981979,SRR1981988,SRR4444961,SRR4444955,SRR1758998,SRR1758996,SRR4444956,SRR4444957,SRR4444958,SRR1758997,SRR4444954,SRR1758995,SRR1758992,ERR1331694,SRR4444960,SRR4444959,ERR1331711,ERR1331704,SRR1200909,SRR636941,SRR636898,SRR1200908,ERR1331715,ERR162228,ERR162229,ERR162240,ERR162246,ERR162251,ERR162267,ERR266373,ERR266376,ERR266380,ERR266381,ERR266394,SRR636846,SRR636847,SRR636848,SRR636896,SRR636897,ERR1331712,ERR1331683,SRR636942,SRR636943,ERR1331707,SRR6261041,SRR6261040,ERR1331709,ERR1331708,ERR1331667,SRR6261042,ERR1331671,SRR1789331,SRR1789326,SRR1789059,SRR1789057,SRR1786019,ERR266370,SRR6293952,SRR6293953,SRR6293961,SRR6293972,SRR6293973,SRR6293975,SRR636914,SRR636915,SRR9066934,SRR636913,SRR9066931,SRR9066929,SRR9066919,SRR9066910,SRR636865,SRR636960,SRR636959,SRR636958,SRR636867,SRR636866,ERR266353,ERR162245,ERR162261,ERR162222,ERR266372,ERR266378,ERR266393,ERR162217,ERR162213,ERR266398,SRR5412205,SRR5412207,SRR5412218,SRR5412210,SRR5412211,SRR5412212,SRR5412213,SRR5412215,SRR5412216,SRR5952126,SRR6206914,SRR5952127,SRR5952128,SRR5952129,SRR5952130,SRR6206899,SRR6206904,SRR5412209,SRR6206909,SRR5412206,SRR449436,ERR1331668,ERR1331710,ERR1331725,SRR306742,SRR306747,SRR306749,SRR306751,SRR306752,SRR306753,SRR306754,SRR306744,SRR449433,SRR449434,SRR449435,SRR2925244,SRR2925212,SRR2925205,SRR2925208,ERR2004720,SRR2925245,SRR2925247,SRR2925248,SRR2925209,SRR2925207,SRR2925206,SRR2925204,SRR2925203,SRR2925202,SRR2925201,SRR2925200,SRR2925199,SRR2925198,SRR2925197,SRR2925196,SRR2925255,ERR2004718,SRR2925249,SRR2925251,SRR2925252,SRR2925253,SRR2925254,ERR2004719,SRR2925258,SRR5115667,SRR5115668,SRR5115669,SRR5115678,SRR2925210,SRR3468375,SRR3468373,SRR3468374,SRR3468369,SRR3468372,SRR636907,SRR636858,SRR636859,SRR636908,SRR636909,SRR636951,SRR636952,SRR636857,SRR636953,SRR1549160,SRR1549162,SRR3468363,SRR3468366,SRR3468367,SRR3468368,SRR5291531,SRR5520663,SRR5291532,SRR5520660,SRR5520661,SRR5520662,SRR3144818,SRR5520664,SRR5520665,SRR5520667,SRR5520668,SRR5520669,SRR3144832,SRR5291530,SRR594442,SRR3144831,SRR1284274,SRR1284266,SRR1284265,SRR1284264,SRR1170173,SRR3144830,SRR954817,SRR594440,SRR594

## per gene ##

In [74]:
show(results1.partitions.X,10,10)

ensembl_id,ENSG00000139990,ENSG00000073921,ENSG00000139687,ENSG00000119977,ENSG00000242866,ENSG00000135506,ENSG00000162426,ENSG00000165995,ENSG00000073756,ENSG00000138050
run,,,,,,,,,,
SRR5961830,26.655,465.005,55.082,13.343,3.845,86.345,7.693,137.115,13.902,52.017
SRR5885325,10.578,95.720,7.969,17.294,0.165,92.622,1.908,3.058,4.627,4.955
SRR306843,11.918,78.732,1.167,4.372,2.623,71.952,13.427,25.618,4.684,1.508
SRR306844,18.714,47.023,13.521,4.307,21.699,254.783,7.247,15.522,0.705,11.868
SRR306845,16.423,61.764,15.757,11.516,25.387,152.155,18.972,23.108,1.287,12.192
SRR306841,12.673,84.034,12.452,13.777,2.591,87.754,9.910,31.474,1.816,7.506
SRR306840,16.837,99.109,14.010,12.868,11.620,84.165,15.426,37.553,3.502,9.418
SRR306839,12.512,104.142,5.173,7.631,4.599,92.039,20.007,29.659,2.401,3.781
SRR787277,26.026,575.293,104.400,10.912,0.199,123.212,0.146,42.191,82.374,40.371


In [26]:
first = results1.gene_details("YTHDC1", data.samples) 
show(first,100,10)

general info: symbol                        YTHDC1
gain_score_to_lifespan    457279.425
kendall_tau_to_lifespan        0.103
Name: ENSG00000083896, dtype: object


,expressions,shap_values,bioproject,series,species,tissue,sample_name,characteristics,sequencer,age,sex,tumor,source,study,study_title,salmon_version,library_layout,library_selection,library_strategy,lib_type,bootstrap,protocol,common_name,animal_class,order,family,lifespan,ensembl_url,mass_g,metabolic_rate,temperature,temperature_kelvin,gestation_days,taxon,female_maturity_days,male_maturity_days,litters_per_year,inter_birth_interval,birth_weight_g,weaning_weight_g,mtGC
run,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SRR1521445,NaN,11.901,PRJNA255754,geo/query/acc.cgi?acc=GSE59612,Homo_sapiens,Brain,GSM1440503,non-neoplastic brain tissue;non-neoplastic bra...,Illumina_HiSeq_2000,NaN,NaN,no,non-neoplastic brain tissue,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,MRI-localized biopsies reveal subtype-specific...,0.14.1,SINGLE,cDNA,RNA-Seq,A,128.000,Total RNA was isolated from the tissue specime...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5008362,NaN,12.053,PRJNA352990,geo/query/acc.cgi?acc=GSE89714,Homo_sapiens,Heart,GSM2387234,Heart;heart;no,Illumina_HiSeq_2000,NaN,NaN,no,heart,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Differential gene expressions in the heart of ...,0.14.1,SINGLE,cDNA,RNA-Seq,A,128.000,Human hypertrophic heart tissue samples were o...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR8702484,NaN,11.851,PRJNA526249,PRJNA526249,Homo_sapiens,Liver,GSM3662889,Liver;Penn Cohort 1;liver;B7;no,Illumina_HiSeq_2500,NaN,NaN,no,liver,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Genetic and Epigenetic Fine Mapping of Complex...,1.2.1,PAIRED,cDNA,RNA-Seq,A,96.000,Illumina TruSeq Stranded mRNA kit QIAGEN's All...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5961972,NaN,11.858,PRJNA398031,PRJNA398031,Homo_sapiens,Brain,GSM2740785,no;no;5.6;no;72;17;Anterior Insula (aINS);yes ...,Illumina_HiSeq_2000,72,female,no,Anterior Insula (aINS),https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Sex-specific Transcriptional Signatures in Hum...,0.14.1,PAIRED,cDNA,RNA-Seq,A,128.000,RNA was extracted using Trizol Reagents from h...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5961875,NaN,11.949,PRJNA398031,PRJNA398031,Homo_sapiens,Brain,GSM2740688,no;yes;7.5;no;55;2.5;Dorsolateral prefrontal c...,Illumina_HiSeq_2000,55,female,no,Dorsolateral prefrontal cortex (dlPFC; BA8/9),https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Sex-specific Transcriptional Signatures in Hum...,0.14.1,PAIRED,cDNA,RNA-Seq,A,128.000,RNA was extracted using Trizol Reagents from h...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5961830,NaN,11.893,PRJNA398031,PRJNA398031,Homo_sapiens,Brain,GSM2740643,no;no;5.7;no;72;17;Orbitofrontal (OFC; BA11);y...,Illumina_HiSeq_2000,72,female,no,Orbitofrontal (OFC; BA11),https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Sex-specific Transcriptional Signatures in Hum...,0.14.1,PAIRED,cDNA,RNA-Seq,A,128.000,RNA was extracted using Trizol Reagents from h...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5885325,NaN,11.791,PRJNA396588,geo/query/acc.cgi?acc=GSE86095_GSE102101,Homo_sapiens,Kidney,GSM2723925,no;adjacent normal kidney;patient 40911432;adj...,Illumina_HiSeq_2000,NaN,NaN,no,adjacent normal kidney,https:/

In [42]:
human = first[(first.species == "Homo_sapiens")]
#human[human.expressions.isna()==False]
human

,expressions,shap_values,bioproject,series,species,tissue,sample_name,characteristics,sequencer,age,sex,tumor,source,study,study_title,salmon_version,library_layout,library_selection,library_strategy,lib_type,bootstrap,protocol,common_name,animal_class,order,family,lifespan,ensembl_url,mass_g,metabolic_rate,temperature,temperature_kelvin,gestation_days,taxon,female_maturity_days,male_maturity_days,litters_per_year,inter_birth_interval,birth_weight_g,weaning_weight_g,mtGC
run,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SRR1521445,NaN,11.901,PRJNA255754,geo/query/acc.cgi?acc=GSE59612,Homo_sapiens,Brain,GSM1440503,non-neoplastic brain tissue;non-neoplastic bra...,Illumina_HiSeq_2000,NaN,NaN,no,non-neoplastic brain tissue,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,MRI-localized biopsies reveal subtype-specific...,0.14.1,SINGLE,cDNA,RNA-Seq,A,128.000,Total RNA was isolated from the tissue specime...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5008362,NaN,12.053,PRJNA352990,geo/query/acc.cgi?acc=GSE89714,Homo_sapiens,Heart,GSM2387234,Heart;heart;no,Illumina_HiSeq_2000,NaN,NaN,no,heart,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Differential gene expressions in the heart of ...,0.14.1,SINGLE,cDNA,RNA-Seq,A,128.000,Human hypertrophic heart tissue samples were o...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR8702484,NaN,11.851,PRJNA526249,PRJNA526249,Homo_sapiens,Liver,GSM3662889,Liver;Penn Cohort 1;liver;B7;no,Illumina_HiSeq_2500,NaN,NaN,no,liver,https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Genetic and Epigenetic Fine Mapping of Complex...,1.2.1,PAIRED,cDNA,RNA-Seq,A,96.000,Illumina TruSeq Stranded mRNA kit QIAGEN's All...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5961972,NaN,11.858,PRJNA398031,PRJNA398031,Homo_sapiens,Brain,GSM2740785,no;no;5.6;no;72;17;Anterior Insula (aINS);yes ...,Illumina_HiSeq_2000,72,female,no,Anterior Insula (aINS),https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Sex-specific Transcriptional Signatures in Hum...,0.14.1,PAIRED,cDNA,RNA-Seq,A,128.000,RNA was extracted using Trizol Reagents from h...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5961875,NaN,11.949,PRJNA398031,PRJNA398031,Homo_sapiens,Brain,GSM2740688,no;yes;7.5;no;55;2.5;Dorsolateral prefrontal c...,Illumina_HiSeq_2000,55,female,no,Dorsolateral prefrontal cortex (dlPFC; BA8/9),https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Sex-specific Transcriptional Signatures in Hum...,0.14.1,PAIRED,cDNA,RNA-Seq,A,128.000,RNA was extracted using Trizol Reagents from h...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5961830,NaN,11.893,PRJNA398031,PRJNA398031,Homo_sapiens,Brain,GSM2740643,no;no;5.7;no;72;17;Orbitofrontal (OFC; BA11);y...,Illumina_HiSeq_2000,72,female,no,Orbitofrontal (OFC; BA11),https://trace.ncbi.nlm.nih.gov/Traces/sra/?stu...,Sex-specific Transcriptional Signatures in Hum...,0.14.1,PAIRED,cDNA,RNA-Seq,A,128.000,RNA was extracted using Trizol Reagents from h...,Human,Mammalia,Primates,Hominidae,122.500,https://www.ensembl.org/Homo_sapiens,70000.000,82.780,37.000,310.150,280.000,taxon#9606,1.000,5110.000,0.300,639.000,3312.500,nan,44.360
SRR5885325,NaN,11.791,PRJNA396588,geo/query/acc.cgi?acc=GSE86095_GSE102101,Homo_sapiens,Kidney,GSM2723925,no;adjacent normal kidney;patient 40911432;adj...,Illumina_HiSeq_2000,NaN,NaN,no,adjacent normal kidney,https:/